# Data Masters: Case

## Bibliotecas

In [1]:
# --- Data Exploration and Viz --- #
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# --- Classification models --- #
from sklearn.ensemble import \
    GradientBoostingClassifier, \
    RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

# --- Pipeline Building --- #
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# --- Model Evaluation --- #
#from resources.custommetrics import profit
from sklearn.metrics import \
    roc_auc_score, \
    recall_score, \
    precision_score, \
    f1_score, \
    confusion_matrix

# --- Tuning --- #
from sklearn.model_selection import \
    GridSearchCV, \
    train_test_split

# --- Preprocessing --- #
from sklearn.preprocessing import \
    StandardScaler, \
    OrdinalEncoder, \
    FunctionTransformer

from sklearn.impute import SimpleImputer
 
from resources.customtransformers import \
    DropConstantColumns, \
    DropDuplicateColumns, \
    AddNoneCount, \
    AddNonZeroCount

# --- Cluster Analysis --- #
#from sklearn.decomposition import PCA
#from sklearn.cluster import KMeans

## Leitura dos dados

In [2]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
train, validation = train_test_split(
    train,
    test_size=0.25,
    random_state=42,
    stratify=train["TARGET"]
)

## Definição dos pipelines de pré-processamento

### Pipeline 1: Mantendo dados originais

Este pipeline serve como um "one size fits all" para os passos seguintes. Como as missing features do conjunto de dados já foram preenchidas artificalmente, tratá-las para modelos de árvore não necessariamente trará melhora na performance, uma vez que o preenchimento é padronizado por prefixo e estes modelos lidam bem com relações não-lineares entre variáveis. Assim, a partir deste pipeline, podemos iniciar os testes de diferentes modelos baseados em árvores.

In [3]:
prep_01 = Pipeline(
    steps=[
        ("dcc", DropConstantColumns()),
        ("ddc", DropDuplicateColumns()),
        (
            "anc_delta",
            AddNoneCount(
                prefix="delta",
                fake_value=9999999999
            )
        ),
        ("anzc_saldo", AddNonZeroCount(prefix="saldo")),
        ("anzc_imp", AddNonZeroCount(prefix="imp")),
        ("anzc_delta", AddNonZeroCount(prefix="delta")),
        ("anzc_ind", AddNonZeroCount(prefix="ind")),
        (
            "col_specific",
            ColumnTransformer(
                [
                    (
                        "ord_encoders",
                        OrdinalEncoder(
                            handle_unknown="use_encoded_value",
                            unknown_value=-1,
                            encoded_missing_value=-1,
                            min_frequency=40
                        ),
                        ["var36","var21"]                    
                    )
                ],
                remainder="passthrough"
            )
        )
    ]
)

### Pipeline 2: Imputando NaN com mediana

No caso das variáveis delta, temos dados extremamente esparsos. Assim, optou-se por imputar os valores faltantes com a mediana, uma vez que a média seria aftetada por outliers. Essencialmente, estamos preenchendo estes valores com 0 e colocando-os no mesmo "grupo" que a maioria das observações.

In [4]:
prep_02 = Pipeline(
    steps=[
        ("dcc", DropConstantColumns()),
        ("ddc", DropDuplicateColumns()),
        (
            "anc_delta",
            AddNoneCount(
                prefix="delta",
                fake_value=9999999999
            )
        ),
        (
            "impute_nan_delta",
            SimpleImputer(
                missing_values=9999999999,
                strategy="median"
            )
        ),
        (
            "impute_nan_var3_transformer",
            ColumnTransformer(
                (
                    "impute_nan_var3",
                    SimpleImputer(
                        missing_values=-999999,
                        strategy="constant",
                        fill_value=None
                    ),
                    "var3"
                ),
                remainder="passthrough"
            )
        ),
        ("anzc_saldo", AddNonZeroCount(prefix="saldo")),
        ("anzc_imp", AddNonZeroCount(prefix="imp")),
        ("anzc_delta", AddNonZeroCount(prefix="delta")),
        ("anzc_ind", AddNonZeroCount(prefix="ind")),
        (
            "col_specific",
            ColumnTransformer(
                [
                    (
                        "ord_encoders",
                        OrdinalEncoder(
                            handle_unknown="use_encoded_value",
                            unknown_value=-1,
                            encoded_missing_value=-1,
                            min_frequency=40
                        ),
                        ["var36","var21"]                
                    )
                ],
                remainder="passthrough"
            )
        )
    ]
)

## Definição da métrica

Os modelos (com excessão de uma árvore simples inicial) passaram por validação cruzada para hiperparametrização buscando maximizar a AUC -- métrica que, de modo geral, indica quão bem o modelo consegue separar as classes da variável "TARGET" ao comparar as estimativas com os valores reais com diferentes cortes de classificação. O corte de classificação, por sua vez, foi ajustado sobre o modelo campeão (com a maior AUC) com base na métrica sugerida pelo enunciado deste trabalho, ou seja, pela soma de falsos positivos * -10 e  verdadeiros positivos * 90, buscando maximizar o lucro do banco sobre a ação sobre um dataset de validação.

In [5]:
def profit(y_true, y_pred_proba, threshold):
    y_pred = (y_pred_proba[:,1] >= threshold).astype(int)
    fp = np.sum((y_pred == 1) & (y_true == 0))
    tp = np.sum((y_pred == 1) & (y_true == 1))
    n = len(y_true)
    return (fp * -10 + tp * 90) / n

## Avaliação dos modelos

### Modelo base: árvore de classificação

Aqui definimos um ponto de partida para os modelos de classificação. Foi escolhida a árvore de classificação pois esta servirá como base dos dois outros algorítmos testados (Random Forest e Gradient Boosting). Este modelo não passa por gridsearch e seu resultado é considerado o mínimo a ser batido pelos outros modelos. 

In [7]:
simple_tree = Pipeline(
    steps=[
        ("preprocessing", prep_01),
        ("clf", DecisionTreeClassifier())
    ]
)

simple_tree

Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('dcc', DropConstantColumns()),
                                 ('ddc', DropDuplicateColumns()),
                                 ('anc_delta',
                                  AddNoneCount(fake_value=9999999999,
                                               prefix='delta')),
                                 ('anzc_saldo',
                                  AddNonZeroCount(prefix='saldo')),
                                 ('anzc_imp', AddNonZeroCount(prefix='imp')),
                                 ('anzc_delta',
                                  AddNonZeroCount(prefix='delta')),
                                 ('anzc_ind', AddNonZeroCount(prefix='ind')),
                                 ('col_specific',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('ord_encoders',
                                                                   OrdinalEncoder(encoded_missing_value=-1,
                                                                                  handle_unknown='use_encoded_value',
                                                                                  min_frequency=40,
                                                                                  unknown_value=-1),
                                                                   ['var36',
                                                                    'var21'])]))])),
                ('clf', DecisionTreeClassifier())])

In [8]:
simple_tree = simple_tree \
    .fit(
        train.drop(["ID","TARGET"], axis=1),
        train["TARGET"]
    )

In [9]:
y_pred = simple_tree \
    .predict_proba(
        validation.drop(["ID","TARGET"], axis=1)
    )

In [ ]:
class FitTuneEval():
    def __init__(self, pipeline, target="TARGET", ignore=[]):
        self.pipeline = pipeline
        self.target = target
        self.ignore = ignore
        pass

    def fit(self, X):
        train, validation = train_test_split(
            train,
            test_size=.25,
            random_state=42,
            stratify=train[self.target]
        )
        self.pipeline = self.pipeline.fit(
            train.drop(self.ignore+self.target, axis=1),
            train[self.target]
        )

In [23]:
y_true = validation["TARGET"]

In [25]:
profit(y_true, y_pred)

0.2170481452249408

In [126]:
def predict_probability(X, model):
    return model.predict_proba(X)[:,1]

def make_result_df(y,y_pred):
    df = y.reset_index()
    df["prob"] = y_pred
    df["pred"] = y_pred
    return df

def predict_class(y, c):
    y["pred"] = y["prob"].apply(lambda x: 1 if x >= c else 0)
    return y

def evaluate(df, c):
    tn, fp, fn, tp = confusion_matrix(
        df["TARGET"],
        predict_class(df, c)["pred"],
    ).ravel()
    return (tn*0+fp*(-10)+fn*0+tp*(100-10))/len(df)

def select_threshold(X, y):
    y_pred = predict_probability(X, gscv)
    df = make_result_df(y,y_pred)

    thresh_scores = [[i/100, 0] for i in range(101)]

    for i in range(len(thresh_scores)):
        c = thresh_scores[i][0]
        thresh_scores[i][1] = evaluate(df,c)

    thresh_scores = pd.DataFrame(thresh_scores,columns=["Threshold", "Profit"])

    best_threshold = thresh_scores.iloc[
        thresh_scores.Profit.idxmax()
    ].Threshold

    return best_threshold